<a href="https://colab.research.google.com/github/darkfapper/Pandoras_Box/blob/main/Bayesian_Neural_Network_Pyro_Regression_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [85]:
!pip install pyro-ppl[funsor] torchsummary uncertainty-toolbox

In [86]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader , Dataset
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import r2_score , mean_squared_error , mean_absolute_error , mean_pinball_loss
import pyro
import pyro.distributions as dist
from pyro.nn import PyroModule, PyroSample
from pyro.infer import SVI, Trace_ELBO , JitTrace_ELBO , Predictive
from pyro.infer.autoguide import (AutoDiagonalNormal , AutoMultivariateNormal ,
 AutoLaplaceApproximation , AutoNormal , AutoNormalizingFlow , AutoRegressiveMessenger,
 AutoNormalMessenger , AutoIAFNormal)
from pyro.distributions.transforms import block_autoregressive, iterated, affine_autoregressive , spline_autoregressive , affine_autoregressive
from pyro.infer.autoguide.gaussian import AutoGaussianFunsor
from tqdm.auto import trange
from scipy import stats as st
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, f1_score , roc_auc_score , brier_score_loss
from sklearn import datasets
from pyro.infer import MCMC, NUTS
import arviz as az
from functools import partial
from pyro.infer.reparam import NeuTraReparam
import pyro.poutine as poutine
from pyro.distributions.transforms import (neural_autoregressive ,spline_autoregressive , affine_autoregressive ,
affine_coupling , planar , conditional_affine_autoregressive, conditional_neural_autoregressive , ConditionalAffineAutoregressive)
torch.set_printoptions(precision=5,sci_mode=False)
# Set random seed for reproducibility
np.random.seed(42)
pyro.set_rng_seed(42)

# from google.colab import drive
# drive.mount('/content/drive')
# df = pd.read_csv("/content/drive/MyDrive/creditcard_2023.csv").drop(columns = ["id"])
# sc_amount = StandardScaler()
# df["Amount"] = sc_amount.fit_transform(df["Amount"].values.reshape(-1,1))
# df
# X = df.drop(columns = ["Class"])
# y = df["Class"]

In [87]:
## Importando os dados
from google import colab
colab.drive.mount('/content/drive')
df = pd.read_csv("/content/sample_data/california_housing_train.csv")
df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-114.31,34.19,15.0,5612.0,1283.0,1015.0,472.0,1.4936,66900.0
1,-114.47,34.40,19.0,7650.0,1901.0,1129.0,463.0,1.8200,80100.0
2,-114.56,33.69,17.0,720.0,174.0,333.0,117.0,1.6509,85700.0
3,-114.57,33.64,14.0,1501.0,337.0,515.0,226.0,3.1917,73400.0
4,-114.57,33.57,20.0,1454.0,326.0,624.0,262.0,1.9250,65500.0
...,...,...,...,...,...,...,...,...,...
16995,-124.26,40.58,52.0,2217.0,394.0,907.0,369.0,2.3571,111400.0
16996,-124.27,40.69,36.0,2349.0,528.0,1194.0,465.0,2.5179,79000.0
16997,-124.30,41.84,17.0,2677.0,531.0,1244.0,456.0,3.0313,103600.0
16998,-124.30,41.80,19.0,2672.0,552.0,1298.0,478.0,1.9797,85800.0


In [88]:
from sklearn.preprocessing import StandardScaler
sc_dict = {}
for col in df.columns:
  sc = StandardScaler()
  df[col] = sc.fit_transform(df[col].values.reshape(-1,1))
  sc_dict[col] = sc

In [89]:
X = df.drop(columns = ["median_house_value"])
y = df["median_house_value"]

In [90]:
X.dtypes

longitude             float64
latitude              float64
housing_median_age    float64
total_rooms           float64
total_bedrooms        float64
population            float64
households            float64
median_income         float64
dtype: object

In [91]:
## Separando em Treino e teste

X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, test_size=0.33, random_state=42)

In [92]:
# Convert data to PyTorch tensors
X_train_mc = torch.from_numpy(X_train).float()
y_train_mc = torch.from_numpy(y_train).float()

X_test_mc = torch.from_numpy(X_test).float()
y_test_mc = torch.from_numpy(y_test).float()

batch_size = 64

In [93]:
trainloader = DataLoader(list(zip(X_train_mc, y_train_mc)), shuffle=True, batch_size=batch_size)
validloader = DataLoader(list(zip(X_train_mc, y_train_mc)), shuffle=True, batch_size=len(X_train_mc))
testloader = DataLoader(list(zip(X_test_mc, y_test_mc)), shuffle=True, batch_size=len(X_test_mc))
loaders_dict = {"train": trainloader, "valid": validloader, "test": testloader}

In [94]:

# Copyright (C) 2023 Intel Labs
#
# BSD-3-Clause License
#
# Redistribution and use in source and binary forms, with or without modification,
# are permitted provided that the following conditions are met:
# 1. Redistributions of source code must retain the above copyright notice,
#    this list of conditions and the following disclaimer.
# 2. Redistributions in binary form must reproduce the above copyright notice,
#    this list of conditions and the following disclaimer in the documentation
#    and/or other materials provided with the distribution.
# 3. Neither the name of the copyright holder nor the names of its contributors
#    may be used to endorse or promote products derived from this software
#    without specific prior written permission.
#
# THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS"
# AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO,
# THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE
# ARE DISCLAIMED.  IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS
# BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY,
# OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT
# OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS;
# OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY,
# WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE
# OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE,
# EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.
#
# Utily functions for variational inference in Bayesian deep neural networks
#
# @authors: Ranganath Krishnan
#
# ===============================================================================================



def entropy(prob):
    return -1 * np.sum(prob * np.log(prob + 1e-15), axis=0)


def predictive_entropy(mc_preds):
    """
    Compute the entropy of the mean of the predictive distribution
    obtained from Monte Carlo sampling during prediction phase.
    """
    return entropy(np.mean(mc_preds, axis=0))


def mutual_information(mc_preds):
    """
    Compute the difference between the entropy of the mean of the
    predictive distribution and the mean of the entropy.
    """
    mutual_info = entropy(np.mean(mc_preds, axis=0)) - np.mean(entropy(mc_preds),
                                                               axis=0)
    return mutual_info

In [95]:
class MLP(nn.Module):
    def __init__(self, input_dim=9, output_dim=1, hidden_dim=48, n_hidden_layers=3, use_dropout=False,p_dropout=0.25):
        super().__init__()

        self.use_dropout = use_dropout
        if use_dropout:
            self.dropout = nn.Dropout(p=p_dropout)
        self.activation = nn.LeakyReLU()

        # dynamically define architecture
        self.layer_sizes = [input_dim] + n_hidden_layers * [hidden_dim] + [output_dim]
        layer_list = [nn.Linear(self.layer_sizes[idx - 1], self.layer_sizes[idx]) for idx in
                      range(1, len(self.layer_sizes))]
        self.layers = nn.ModuleList(layer_list)

    def forward(self, input):
        # input = input.reshape(1, -1)
        hidden = self.activation(self.layers[0](input))
        for layer in self.layers[1:-1]:
            hidden_temp = self.activation(layer(hidden))

            if self.use_dropout:
                hidden_temp = self.dropout(hidden_temp)

            hidden = hidden_temp + hidden  # residual connection

        output_mean = self.layers[-1](hidden).squeeze()
        return output_mean

In [96]:
class BNN(PyroModule):
    def __init__(self, in_dim=X.shape[1], out_dim=1, hid_dim=48, n_hid_layers=3, prior_scale=10.):
        super().__init__()

        self.activation = nn.LeakyReLU()  # could also be ReLU or LeakyReLU
        assert in_dim > 0 and out_dim > 0 and hid_dim > 0 and n_hid_layers > 0  # make sure the dimensions are valid

        # Define the layer sizes and the PyroModule layer list
        self.layer_sizes = [in_dim] + n_hid_layers * [hid_dim] + [out_dim]
        layer_list = [PyroModule[nn.Linear](self.layer_sizes[idx - 1], self.layer_sizes[idx]) for idx in
                      range(1, len(self.layer_sizes))]
        self.layers = PyroModule[torch.nn.ModuleList](layer_list)

        for layer_idx, layer in enumerate(self.layers):
            layer.weight = PyroSample(dist.Normal(0., prior_scale * np.sqrt(2 / self.layer_sizes[layer_idx])).expand(
                [self.layer_sizes[layer_idx + 1], self.layer_sizes[layer_idx]]).to_event(2))
            layer.bias = PyroSample(dist.Normal(0., prior_scale).expand([self.layer_sizes[layer_idx + 1]]).to_event(1))

    def forward(self, x, y=None):
        # x = x.reshape(x.shape[0], -1)
        x = self.activation(self.layers[0](x))  # input --> hidden
        for layer in self.layers[1:-1]:
            x = self.activation(layer(x))  # hidden --> hidden
        mu = self.layers[-1](x).squeeze()  # hidden --> output
        yhat = torch.nn.functional.sigmoid(mu)
        sigma = pyro.sample("sigma", dist.Gamma(.5, 1))
            ## Incerteza Aletorica
        with pyro.plate("data", x.shape[0]):
            obs = pyro.sample("obs", dist.Normal(mu, sigma * sigma), obs=y)
        return mu

In [97]:
# # for i in range(X.shape[1]):
# #   X[:,i] = (X[:,i] - np.mean(X[:,i])) / np.std(X[:,i])

# sc_dict = {}
# for col in X_df.columns:
#   sc = StandardScaler()
#   X_df[col] = sc.fit_transform(X_df[col].values.reshape(-1,1))
#   sc_dict[col] = sc

In [98]:
#Funções auxiliares

def entropy(prob):
    return -1 * np.sum(prob * np.log(prob + 1e-15), axis=0)


def without_keys(d, keys):
    return {k: v for k, v in d.items() if k not in keys}



def func_metrics(samples, y):
  if isinstance(y,torch.Tensor):
    y = y.detach().numpy()
  if isinstance(samples,dict):
    ## Processando as Amostras
    samples = samples["obs"].detach().numpy()
    mean = y_pred = np.mean(samples,axis = 0)
    std = np.std(samples["obs"].detach().numpy(),axis = 0)
  else:
    _preds_probs = samples
    mean = np.mean(_preds_probs,axis = 0)
  R2 = r2_score(y, y_pred)
  MSE = mean_squared_error(y, y_pred)
  RMSE = np.sqrt(MSE)
  MAE = mean_absolute_error(y, y_pred)
  pred_entropy = predictive_entropy(samples)
  mutual_info = mutual_information(samples)
  PINBALL = mean_pinball_loss(y, y_pred)
  return pd.DataFrame({"R2":R2, "MSE":MSE, "RMSE":RMSE, "MAE":MAE, "PINBALL":PINBALL, "ENTROPY":ENTROPY},index = [0])

def variational_train(model,guide_,loaders_dict,epochs = 5,num_samples = 150):
  pyro.clear_param_store()
  guide = guide_(model)
  optimizer = pyro.optim.Adam({"lr": 0.001})
  svi = SVI(model, guide, optimizer, loss=Trace_ELBO())

  progress_bar = trange(epochs)
  losses = []
  for epoch in progress_bar:
    for X_batch, y_batch in loaders_dict["train"]:
      loss = svi.step(X_batch, y_batch)
      loss = loss/len(loaders_dict["train"])
      losses.append(loss)
    progress_bar.set_description(f"ELBO: {loss}")
  ## Filtrando Outliers do Gráfico
  losses_ = np.array(losses)
  z = np.abs(st.zscore(losses_))
  losses_ = losses_[z < 2]
  plt.plot(range(1,len(losses_)+1),losses_)
  plt.xlabel("Epochs")
  plt.ylabel("ELBO")
  plt.show()

  ## Avaliação da Predição

  list_metrics = []
  predictive = Predictive(model, guide=guide,num_samples=num_samples)
  for name_set, loader in without_keys(loaders_dict,"train").items():
    X , y = next(iter(loader))
    # X = X.cuda()
    samples = predictive(X)
    metrics = func_metrics(samples,y)
    metrics["guide"] = str(guide)
    metrics["model"] = str(model)
    metrics["name_set"] = name_set
    list_metrics.append(metrics)
  metrics_full = pd.concat(list_metrics,axis = 0)

  return {"predictive":predictive ,"metrics":metrics_full ,"hist_train":losses}

def neural_mcmc_train(model,loaders_dict,guide,epochs = 50,num_samples = 25,warmup_steps = 10 ):
  pyro.clear_param_store()
  # guide = AutoIAFNormal(model,num_transforms=num_transforms)
  optimizer = pyro.optim.Adam({"lr": 0.001})
  svi = SVI(model, guide, optimizer, loss=Trace_ELBO())

  progress_bar = trange(epochs)
  losses = []
  for epoch in progress_bar:
    for X_batch, y_batch in loaders_dict["train"]:
      loss = svi.step(X_batch, y_batch)
      loss = loss/len(loaders_dict["train"])
      losses.append(loss)
  X_mc , y_mc = next(iter(loaders_dict["valid"]))
  neutra = NeuTraReparam(guide.requires_grad_(False))
  #neutra = NeuTraReparam(guide)
  model = poutine.reparam(model,config=lambda _: neutra)
  nuts_kernel = NUTS(model, jit_compile=False)
  mcmc = MCMC(nuts_kernel, num_samples=num_samples,warmup_steps=warmup_steps)
  mcmc.run(X_mc, y_mc)
  print(mcmc.summary(p = 0.5))
  predictive = Predictive(model, posterior_samples=mcmc.get_samples(num_samples))

  posterior_samples = mcmc.get_samples(num_samples)
  posterior_predictive = Predictive(model, posterior_samples)(X_mc)
  prior = Predictive(model, num_samples=num_samples)(X_mc)
  pyro_data = az.from_pyro(mcmc,prior=prior,posterior_predictive=posterior_predictive)
  az.plot_trace(pyro_data)

  ## Avaliação da Predição

  list_metrics = []
  predictive = Predictive(model, guide=guide,num_samples=num_samples)
  for name_set, loader in without_keys(loaders_dict,"train").items():
    X , y = next(iter(loader))
    samples = predictive(X)
    metrics = func_metrics(samples,y)
    metrics["guide"] = str(guide)
    metrics["model"] = str(model)
    metrics["name_set"] = name_set
    list_metrics.append(metrics)
  metrics_full = pd.concat(list_metrics,axis = 0)

  return {"predictive":predictive ,"metrics":metrics_full ,"viz_data": pyro_data,"hist_train":losses}



def mcmc_train(model,X,y,arviz = True,num_samples = 25,warmup_steps = 10):

  pyro.clear_param_store()

  nuts_kernel = NUTS(model, jit_compile=True)
  mcmc = MCMC(nuts_kernel, num_samples=num_samples,warmup_steps=warmup_steps)
  mcmc.run(X, y)
  predictive = Predictive(model, posterior_samples=mcmc.get_samples(num_samples))

  posterior_samples = mcmc.get_samples(num_samples)
  posterior_predictive = Predictive(model, posterior_samples)(X)
  prior = Predictive(model, num_samples=num_samples)(X)
  pyro_data = az.from_pyro(mcmc,prior=prior,posterior_predictive=posterior_predictive)
  az.plot_trace(pyro_data)

  ## Avaliação da Predição

  list_metrics = []
  predictive = Predictive(model, guide=guide,num_samples=num_samples)
  for name_set, loader in without_keys(loaders_dict,"train").items():
    X , y = next(iter(loader))
    samples = predictive(X)
    metrics = func_metrics(samples,y)
    metrics["guide"] = str(guide)
    metrics["model"] = str(model)
    metrics["name_set"] = name_set
    list_metrics.append(metrics)
  metrics_full = pd.concat(list_metrics,axis = 0)

  return {"predictive":predictive ,"metrics":metrics_full ,"viz_data": pyro_data,"hist_train":None}


def deterministic_train(net, loaders_dict):
    optimizer = torch.optim.Adam(params=net.parameters(), lr=1e-3)
    criterion = nn.MSELoss()
    losses = []
    progress_bar = trange(500)
    for _ in progress_bar:
      for x_train, y_train in loaders_dict["train"]:
        optimizer.zero_grad()
        loss = criterion(y_train.float(), net(x_train))
        progress_bar.set_postfix(loss=f'{loss / x_train.shape[0]:.3f}')
        loss.backward()
        optimizer.step()
        losses.append(loss)

    ## Avaliação da Predição

    net.eval()
    list_metrics = []
    for name_set, loader in without_keys(loaders_dict,"train").items():
      X , y = next(iter(loader))
      preds = net(X)
      metrics = func_metrics(preds,y)
      metrics["model"] = str(net)
      metrics["name_set"] = name_set
      metrics["guide"] = "deterministic"
      list_metrics.append(metrics)
    metrics_full = pd.concat(list_metrics,axis = 0)



    return {"predictive": net , "metrics":metrics_full,"hist_train": losses}

In [99]:
list_results = []

In [ ]:
## Baseline
model = BNN()
guide = AutoMultivariateNormal
aux = variational_train(model,guide,loaders_dict,epochs = 25,num_samples = 150)
# aux = neural_mcmc_train(model,loaders_dict)
aux["metrics"]
list_results.append(aux)

  0%|          | 0/25 [00:00<?, ?it/s]

In [ ]:
## Baseline
model = BNN()
guide = AutoDiagonalNormal
aux = variational_train(model,guide,loaders_dict,epochs = 25,num_samples = 150)
#aux = neural_mcmc_train(model,loaders_dict)
aux["metrics"]
list_results.append(aux)

In [ ]:
model = BNN()
transform_init = partial(iterated,1,affine_autoregressive,stable = True)
guide = partial(AutoNormalizingFlow,init_transform_fn = transform_init)
aux = variational_train(model,guide,loaders_dict,epochs = 25,num_samples = 150)
print(aux["metrics"])
list_results.append(aux)

In [ ]:
model = BNN()
guide = partial(AutoIAFNormal,num_transforms = 3)
aux = variational_train(model,guide,loaders_dict,epochs = 25,num_samples = 150)
print(aux["metrics"])
list_results.append(aux)

In [ ]:
model = BNN()
transform_init = partial(iterated,1,neural_autoregressive,width = 6)
guide = partial(AutoNormalizingFlow,init_transform_fn = transform_init)
aux = variational_train(model,guide,loaders_dict,epochs = 25,num_samples = 150)
aux["metrics"]
list_results.append(aux)

In [ ]:
model = BNN()
transform_init = partial(iterated,1,planar)
guide = partial(AutoNormalizingFlow,init_transform_fn = transform_init)
aux = variational_train(model,guide,loaders_dict,epochs = 50,num_samples = 150)
print(aux["metrics"])
list_results.append(aux)

In [ ]:
model = BNN()
transform_init = partial(iterated,2,affine_coupling)
guide = partial(AutoNormalizingFlow,init_transform_fn = transform_init)
aux = variational_train(model,guide,loaders_dict,epochs = 50,num_samples = 150)
print(aux["metrics"])
list_results.append(aux)

In [ ]:
pd.DataFrame(list_results[0])

In [ ]:
# from pyro.distributions.transforms import (neural_autoregressive ,spline_autoregressive , affine_autoregressive ,
# affine_coupling , planar , conditional_affine_autoregressive, conditional_neural_autoregressive , ConditionalAffineAutoregressive)
# input_dim = 19
# model = BNN()
# transform_init = partial(iterated,4,affine_autoregressive,stable = True)
# guide = partial(AutoNormalizingFlow,init_transform_fn = transform_init)
# res_dict = {"spline_autoregressive":{"base_fn":spline_autoregressive,"count_bins":4},
#             "affine_coupling":{"base_fn":affine_coupling,"hidden_dims":[20*input_dim]},
#             "neural_autoregressive":{"base_fn":neural_autoregressive,"width":8},
#             "affine_autoregressive":{"base_fn":affine_autoregressive,"stable" : True},
#             "conditional_affine_autoregressive":{"base_fn":conditional_affine_autoregressive,
#                                                  "context_dim": input_dim},}

# aux = variational_train(model,guide,loaders_dict,epochs = 250,num_samples = 150)
# aux["metrics"]

In [ ]:
ensemble_size = 3
ensemble = [MLP(hidden_dim=50, n_hidden_layers=5,input_dim = X_train.shape[1]) for _ in range(ensemble_size)]
ensemble = [deterministic_train(net, loaders_dict) for net in ensemble]


In [ ]:
## Implementação usando Ensemble de Modelos
## entropia preditiva muito menor , repostas muito parecidas
ensemble_preds = [net(X_test_mc).clone().detach() for net in ensemble]
ensemble_preds = np.clip(torch.stack(ensemble_preds, dim=0).detach().numpy(),0,1)
mean = np.mean(ensemble_preds, axis=0)
_preds = np.where(mean > 0.5, 1, 0)
pred_entropy = predictive_entropy(ensemble_preds.T)
mutual_info = mutual_information(ensemble_preds.T)
acc = accuracy_score(y_test_mc,_preds)
f1 = f1_score(y_test_mc,_preds)
auc = roc_auc_score(y_test_mc,mean)
pd.DataFrame({"accuracy":acc,"f1":f1,"auc":auc,"predictive_entropy":pred_entropy,"mutual_information":mutual_info},index = [0])


In [ ]:
## Implementação usando Dropout

net_dropout = MLP(hidden_dim=30, n_hidden_layers=2, use_dropout=True)
net_dropout = deterministic_train(net_dropout, loaders_dict)

In [ ]:
n_dropout_samples = 100

# compute predictions, resampling dropout mask for each forward pass
dropout_preds = [net_dropout(X_train_mc).clone().detach().numpy() for _ in range(n_dropout_samples)]
dropout_preds = np.clip(np.array(dropout_preds),0,1)
mean = np.mean(dropout_preds, axis=0)
_preds = np.where(mean > 0.5, 1, 0)
pred_entropy = predictive_entropy(dropout_preds.T)
mutual_info = mutual_information(dropout_preds.T)
acc = accuracy_score(y_train_mc,_preds)
f1 = f1_score(y_train_mc,_preds)
auc = roc_auc_score(y_train_mc,mean)
result = pd.DataFrame({"accuracy":acc,"f1":f1,"auc":auc,"predictive_entropy":pred_entropy,"mutual_information":mutual_info},index = [0])
list_results.append(result)

In [ ]:
df_results = pd.DataFrame(list_results)
df_results

In [ ]:
## Criar um codigo para descompactar a lista de resultados
for result in list_results:
  print(result)